In [1]:
"""
#textに含まれる単語をkeyとしその単語の個数をvalueとした辞書を返す.
def get_mecab_text(text):
    word = {}
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        if (fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞'):
            w = node.surface
            word[w] = word.get(w, 0) + 1
        node = node.next
    return word
"""

'\n#textに含まれる単語をkeyとしその単語の個数をvalueとした辞書を返す.\ndef get_mecab_text(text):\n    word = {}\n    node = mt.parseToNode(text)\n    while node:\n        fields = node.feature.split(",")\n        if (fields[0] == \'名詞\' or fields[0] == \'動詞\' or fields[0] == \'形容詞\'):\n            w = node.surface\n            word[w] = word.get(w, 0) + 1\n        node = node.next\n    return word\n'

# word2vecとdoc2vecをそれぞれ学習させる

In [2]:
from gensim.models import word2vec
import MeCab
import numpy as np
import json
import hashlib
from sklearn.cross_decomposition import CCA
from googletrans import Translator

In [3]:
translator = Translator()
model = word2vec.Word2Vec.load("wiki_plus.model")
mt = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd/')
mt.parse('')

'EOS\n'

In [4]:
from gensim.models.doc2vec import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
model_doc = Doc2Vec.load("jawiki.doc2vec.dbow300d.model")

# wikipediaから都道府県についての文章を拾ってきてデータフレーム化

In [5]:
import codecs
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://ja.wikipedia.org/wiki/"
keyword_list = ["北海道","青森県","岩手県","宮城県","秋田県","山形県","福島県","茨城県",
                "栃木県","群馬県","埼玉県","千葉県","東京都","神奈川県","新潟県","富山県",
                "石川県","福井県","山梨県","長野県","岐阜県","静岡県","愛知県","三重県",
                "滋賀県","京都府","大阪府","兵庫県","奈良県","和歌山県","鳥取県","島根県",
                "岡山県","広島県","山口県","徳島県","香川県","愛媛県","高知県","福岡県",
                "佐賀県","長崎県","熊本県","大分県","宮崎県","鹿児島県","沖縄県"]

training_docs = []
df = pd.DataFrame(columns=['text'])
for keyword in keyword_list:

    response = requests.get(url + keyword)
    # htmlに変換
    html = response.text
    
    soup = BeautifulSoup(html, 'lxml')
    corpus = []
    for p_tag in soup.find_all('p'):
        text = "".join(p_tag.text.strip().split(" "))
        if len(text) == 0:
            continue
        # e.g. [注釈9] -> ''
        text = re.sub(r"\[注釈[0-9]+\]", "", text)
        # e.g. [20] -> ''
        text = re.sub(r"\[[0-9]+\]", "", text)
        corpus.append(text)
    corpus = ''.join(corpus)
    df.loc[keyword] = corpus
df

/Users/wsiny/.pyenv/versions/3.8.2/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


,text
北海道,■―政令指定都市/■―市/■―町/■―村地理院地図\nGoogle\nBing\nGeoHa...
青森県,■―市/■―町/■―村地理院地図\nGoogle\nBing\nGeoHack\nMapFa...
岩手県,■―市/■―町/■―村地理院地図\nGoogle\nBing\nGeoHack\nMapFa...
宮城県,■―政令指定都市/■―市/■―町/■―村地理院地図\nGoogle\nBing\nGeoHa...
秋田県,■―市/■―町/■―村地理院地図\nGoogle\nBing\nGeoHack\nMapFa...
山形県,■―市/■―町/■―村地理院地図\nGoogle\nBing\nGeoHack\nMapFa...
福島県,■―市/■―町/■―村地理院地図\nGoogle\nBing\nGeoHack\nMapFa...
茨城県,■―市/■―町/■―村地理院地図\nGoogle\nBing\nGeoHack\nMapFa...
栃木県,■―市/■―町地理院地図\nGoogle\nBing\nGeoHack\nMapFan\nM...
群馬県,■―市/■―町/■―村地理院地図\nGoogle\nBing\nGeoHack\nMapFa...


# testを分かち書きに変換

In [6]:
#get_tags_for_doc2vec('私はサッカー選手です.')出力['私', 'は', 'サッカー選手', 'です', '。']
def get_tags_for_doc2vec(text):
    word = []
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        if node.surface in model.wv:
            w = node.surface
            word.append(w)
        node = node.next
    return word

In [7]:
print(df.loc['広島県','text'][:300])

鳥取県、島根県、岡山県、山口県、愛媛県、※香川県■―政令指定都市/■―市/■―町地理院地図
Google
Bing
GeoHack
MapFan
Mapion
Yahoo!
NAVITIME
ゼンリン広島県（ひろしまけん、英:HiroshimaPrefecture）は、日本の中国地方に位置する県。県庁所在地及び最大の都市は広島市。広島都市圏と備後都市圏を中心に工業（自動車産業、製鉄、造船）・商業が盛ん。県庁所在地の広島市は中国・四国地方最大の都市であり、政令指定都市に指定されている。一方で海・山の豊富な自然にも恵まれ、農業・漁業も盛んである。このため「日本国の縮図」とも呼ばれている。旧国制に倣


In [8]:
df.loc['広島県','text'].count('宮島')

1

In [9]:
for index, row in df.iterrows():
    print((index,row['text'].count('宮島')))

('北海道', 0)
('青森県', 0)
('岩手県', 0)
('宮城県', 0)
('秋田県', 0)
('山形県', 0)
('福島県', 0)
('茨城県', 0)
('栃木県', 0)
('群馬県', 0)
('埼玉県', 0)
('千葉県', 0)
('東京都', 0)
('神奈川県', 0)
('新潟県', 0)
('富山県', 0)
('石川県', 0)
('福井県', 0)
('山梨県', 0)
('長野県', 0)
('岐阜県', 0)
('静岡県', 0)
('愛知県', 0)
('三重県', 0)
('滋賀県', 0)
('京都府', 0)
('大阪府', 0)
('兵庫県', 0)
('奈良県', 0)
('和歌山県', 0)
('鳥取県', 0)
('島根県', 0)
('岡山県', 0)
('広島県', 1)
('山口県', 0)
('徳島県', 0)
('香川県', 0)
('愛媛県', 0)
('高知県', 0)
('福岡県', 0)
('佐賀県', 0)
('長崎県', 0)
('熊本県', 0)
('大分県', 0)
('宮崎県', 0)
('鹿児島県', 0)
('沖縄県', 0)


In [10]:
print(get_tags_for_doc2vec(df.loc['広島県','text'])[:100])

['鳥取県', '、', '島根県', '、', '岡山県', '、', '山口県', '、', '愛媛県', '、', '※', '香川県', '■', '―', '政令指定都市', '市', '町', '地理院地図', 'Google', 'Bing', 'MapFan', 'Mapion', 'Yahoo!', 'NAVITIME', 'ゼンリン', '広島県', '（', 'ひろしま', 'けん', '、', '英', ':', '）', 'は', '、', '日本', 'の', '中国地方', 'に', '位置', 'する', '県', '。', '県庁所在地', '及び', '最大', 'の', '都市', 'は', '広島市', '。', '広島都市圏', 'と', '備後都市圏', 'を', '中心', 'に', '工業', '（', '自動車産業', '、', '製鉄', '、', '造船', '）', '・', '商業', 'が', '盛ん', '。', '県庁所在地', 'の', '広島市', 'は', '中国・四国地方', '最大', 'の', '都市', 'で', 'あり', '、', '政令指定都市', 'に', '指定', 'さ', 'れ', 'て', 'いる', '。', '一方', 'で', '海', '・', '山', 'の', '豊富', 'な', '自然', 'に', 'も']


# これらの文章を追加で学習させる

In [11]:
'''
def get_doc2vec_material_list(text_list):
    words_list = []
    for i in text_list:
        words_list.extend(get_tags_for_doc2vec(i))
    return words_list
'''
'''
#追加学習
training_docs = []
for i, v in df.iterrows():
    text = get_doc2vec_material_list(v['text']) 
    training_docs.append(TaggedDocument(words=text,tags=[i]))
'''
#model_doc.build_vocab(training_docs, update=True)
#model_doc.train(training_docs, total_examples=model.corpus_count, epochs=model.epochs)

"\n#追加学習\ntraining_docs = []\nfor i, v in df.iterrows():\n    text = get_doc2vec_material_list(v['text']) \n    training_docs.append(TaggedDocument(words=text,tags=[i]))\n"

In [12]:
#textに含まれる単語をkeyとしその単語の個数をvalueとした辞書を返す.
def get_tags(text):
    word = {}
    node = mt.parseToNode(text)
    while node:
        fields = node.feature.split(",")
        # 名詞、動詞、形容詞のいずれか、かつ、w2vに登録されている単語だけ
        if (fields[0] == '名詞' or fields[0] == '動詞' or fields[0] == '形容詞') and node.surface in model.wv:
#        if (fields[0] == '名詞' or fields[0] == '形容詞') and node.surface in model.wv:
            w = node.surface
            word[w] = word.get(w, 0) + 1
        node = node.next
    return word

In [13]:
class Event:
    def __init__(self, id, type, score, desc, links):
        self.id = id
        self.type = type
        self.score = score
        self.desc = desc
        self.tags = get_tags(desc)
        self.links = links
    
    def __str__(self):
        return 'id: {id} type: {type} score: {score} tags: {tags}'.format(id=self.id, type=self.type, score=self.score, tags=str(self.tags))

    def __repr__(self):
        return 'id: {id} type: {type} score: {score} tags: {tags}'.format(id=self.id, type=self.type, score=self.score, tags=str(self.tags))

In [14]:
class Event_doc2vec(Event):
    def __init__(self, id, type, score, desc, links):
        super().__init__(id,type,score,desc,links)
        self.tags = get_tags_for_doc2vec(desc)

# eventsの中身

In [15]:
events = {i: Event(i, [], 0.0, v['text'], []) for i, v in df.iterrows()}

In [16]:
events_doc2vec = {i: Event_doc2vec(i, [], 0.0, v['text'], []) for i, v in df.iterrows()}

In [17]:
# cos similality
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2)) #ベクトルの積/（ベクトルの大きさ＊ベクトルの大きさ）

# doc vec with weighted mean
def weighted_mean_vec(e):
    v = np.zeros(model.vector_size)
    s = 1.0
    for w,weight in e.tags.items():
        v += weight * model.wv[w]  #Eventクラスeの単語wの個数＊単語wのベクトル
        s += weight
    return v / s

# doc間のsimilality (weighted mean_vec and cos_sim)
def event_sim_cos(e1, e2):
    return cos_sim(weighted_mean_vec(e1), weighted_mean_vec(e2))

In [18]:
# CCAの相関係数によるsimilality
def get_vectors(e1):
    return np.array([model.wv[w] for w,weight in e1.tags.items()]) # Eventクラスeの単語それぞれのベクトルを要素とする配列
def event_sim_cca(e1, e2):
    x1 = get_vectors(e1)
    x2 = get_vectors(e2)
    n = min(x1.shape[0], x2.shape[0])
    assert(n >= 1)
    cca = CCA(n_components=n)
    cca.fit(x1.transpose(), x2.transpose())
    return cca.score(x1.transpose(), x2.transpose())

In [19]:
def tokenize(text):
    return mt.parse(text).strip().split()

In [20]:
#eventsはidをkeyそのidに対するeventをvalueとした辞書
def search_events_cos(keywords, events):
    e1 = Event('keywords', [], 0.0, keywords, []) #Event(id, type, score, desc, links)
    if len(e1.tags) == 0:
        raise NameError('Tag is empty')
    sims = [(x.id, event_sim_cos(e1, x)) for k,x in events.items() if len(x.tags) >= 1]
    return sorted(sims, key=lambda x:x[1], reverse=True)[:10]

def search_events_cca(keywords, events):
    e1 = Event('keywords', [], 0.0, keywords, [])
    if len(e1.tags) == 0:
        raise NameError('Tag is empty')
    sims = [(x.id, event_sim_cca(e1, x)) for k,x in events.items() if len(x.tags) >= 1]
    return sorted(sims, key=lambda x:x[1], reverse=True)[:10]

In [21]:
#['都道府県']と['keyword']の類似度
def search_events_doc2vec1(keywords,events):
    e1 = Event_doc2vec('keywords',[],0.0,keywords,[])
    if len(e1.tags) == 0:
        raise NameError('Tag is empty')
    sims = [( i, model_doc.docvecs.similarity_unseen_docs(model_doc, [i], e1.tags, alpha=1, min_alpha=0.0001, steps=5)) for i in keyword_list]
    return sorted(sims,key=lambda x:x[1], reverse=True)[:10]

In [22]:
#['都道府県']のtextと['keyword']の類似度
def search_events_doc2vec2(keywords,events):
    e1 = Event_doc2vec('keywords',[],0.0,keywords,[])
    if len(e1.tags) == 0:
        raise NameError('Tag is empty')
    
    sims = [( i, model_doc.docvecs.similarity_unseen_docs(model_doc, events[i].tags, e1.tags, alpha=1, min_alpha=0.0001, steps=5)) for i in keyword_list]
    return sorted(sims,key=lambda x:x[1], reverse=True)[:10]

In [23]:
#infervec (cos) ['都道府県']と['keyword']の類似度
def search_events_doc2vec3(keywords,events):
    e1 = Event_doc2vec('keywords',[],0.0,keywords,[])
    if len(e1.tags) == 0:
        raise NameError('Tag is empty')
    x = model_doc.infer_vector(e1.tags)
    sims = [( i,cos_sim(x,model_doc.infer_vector([i]))) for i in keyword_list]
    return sorted(sims,key=lambda x:x[1], reverse=True)[:10]

In [24]:
#infervec (cos) ['都道府県']のtextと['keyword']の類似度
def search_events_doc2vec4(keywords,events):
    e1 = Event_doc2vec('keywords',[],0.0,keywords,[])
    if len(e1.tags) == 0:
        raise NameError('Tag is empty')
    x = model_doc.infer_vector(e1.tags)
    sims = [( i,cos_sim(x,model_doc.infer_vector(events[i].tags))) for i in keyword_list]
    return sorted(sims,key=lambda x:x[1], reverse=True)[:10]

In [25]:
#['都道府県']と['keyword']の類似度
search_events_doc2vec1('日本三景のひとつに数えられる「宮島」。島内の「嚴島（いつくしま）神社」や「弥山（みせん）原始林」などは、世界文化遺産に登録されています。自然あふれる宮島には野生動物も多く住み、ニホンジカやタヌキ、野鳥などが。また、グルメも宮島観光ではずせません。名物・もみじ饅頭はもちろん、宮島二大グルメの牡蠣とあなごは必食！',events_doc2vec)

[('広島県', 0.3004176),
 ('奈良県', 0.2991788),
 ('福井県', 0.27795634),
 ('岩手県', 0.27440834),
 ('熊本県', 0.27186087),
 ('佐賀県', 0.2528845),
 ('京都府', 0.24311915),
 ('静岡県', 0.24035113),
 ('大阪府', 0.2392287),
 ('沖縄県', 0.23765428)]

In [26]:
#['都道府県']と['keyword']の類似度
search_events_doc2vec1('宮島',events_doc2vec)

[('鳥取県', 0.42152783),
 ('愛媛県', 0.41951495),
 ('岡山県', 0.4191045),
 ('鹿児島県', 0.41488937),
 ('広島県', 0.41213086),
 ('高知県', 0.39465415),
 ('福井県', 0.39428535),
 ('佐賀県', 0.38680696),
 ('山形県', 0.3805033),
 ('福島県', 0.3745277)]

In [27]:
search_events_doc2vec1('一番高い山',events_doc2vec)

[('長野県', 0.43155193),
 ('大阪府', 0.3767528),
 ('佐賀県', 0.35278967),
 ('三重県', 0.31480435),
 ('埼玉県', 0.31451747),
 ('徳島県', 0.3016706),
 ('山口県', 0.29994947),
 ('高知県', 0.29988006),
 ('香川県', 0.29928073),
 ('秋田県', 0.29859316)]

In [28]:
#['都道府県']のtextと['keyword']の類似度
search_events_doc2vec2('日本三景のひとつに数えられる「宮島」。島内の「嚴島（いつくしま）神社」や「弥山（みせん）原始林」などは、世界文化遺産に登録されています。自然あふれる宮島には野生動物も多く住み、ニホンジカやタヌキ、野鳥などが。また、グルメも宮島観光ではずせません。名物・もみじ饅頭はもちろん、宮島二大グルメの牡蠣とあなごは必食！',events_doc2vec)

[('島根県', 0.33329484),
 ('愛媛県', 0.3332015),
 ('広島県', 0.3054567),
 ('和歌山県', 0.30362657),
 ('静岡県', 0.29035488),
 ('宮崎県', 0.28369102),
 ('大分県', 0.27330014),
 ('茨城県', 0.27015796),
 ('奈良県', 0.26974738),
 ('山形県', 0.25560072)]

In [29]:
#['都道府県']のtextと['keyword']の類似度
search_events_doc2vec2('宮島',events_doc2vec)

[('広島県', 0.26500547),
 ('鳥取県', 0.26070568),
 ('千葉県', 0.25379583),
 ('愛媛県', 0.24513507),
 ('秋田県', 0.24155419),
 ('山口県', 0.23970139),
 ('新潟県', 0.23737882),
 ('佐賀県', 0.21947294),
 ('宮崎県', 0.2141827),
 ('徳島県', 0.21369517)]

In [30]:
search_events_doc2vec2('一番高い山',events_doc2vec)

[('愛知県', 0.29419804),
 ('富山県', 0.284406),
 ('山形県', 0.28104743),
 ('岩手県', 0.27918416),
 ('長野県', 0.27660987),
 ('青森県', 0.2737625),
 ('島根県', 0.26159284),
 ('秋田県', 0.25850445),
 ('奈良県', 0.25741735),
 ('静岡県', 0.2569649)]

In [31]:
#infer_vector (cos) ['都道府県']と['keyword']の類似度
search_events_doc2vec3('日本三景のひとつに数えられる「宮島」。島内の「嚴島（いつくしま）神社」や「弥山（みせん）原始林」などは、世界文化遺産に登録されています。自然あふれる宮島には野生動物も多く住み、ニホンジカやタヌキ、野鳥などが。また、グルメも宮島観光ではずせません。名物・もみじ饅頭はもちろん、宮島二大グルメの牡蠣とあなごは必食！',events_doc2vec)

[('広島県', 0.45418555),
 ('和歌山県', 0.42783257),
 ('長崎県', 0.4246859),
 ('山口県', 0.42035398),
 ('奈良県', 0.41969222),
 ('佐賀県', 0.4183242),
 ('熊本県', 0.4171723),
 ('沖縄県', 0.41595522),
 ('愛媛県', 0.41529116),
 ('福井県', 0.41435584)]

In [32]:
#infer_vector (cos) ['都道府県']と['keyword']の類似度
search_events_doc2vec3('宮島',events_doc2vec)

[('広島県', 0.7393561),
 ('山口県', 0.72904325),
 ('福岡県', 0.70890516),
 ('愛媛県', 0.7084612),
 ('北海道', 0.70374954),
 ('島根県', 0.70323914),
 ('高知県', 0.69154096),
 ('香川県', 0.69091076),
 ('福井県', 0.690134),
 ('兵庫県', 0.6893928)]

In [33]:
search_events_doc2vec3('一番高い山',events_doc2vec)

[('秋田県', 0.61598533),
 ('長野県', 0.612996),
 ('山梨県', 0.60960037),
 ('東京都', 0.60602474),
 ('青森県', 0.6057484),
 ('富山県', 0.6040267),
 ('福岡県', 0.6027289),
 ('愛媛県', 0.60176086),
 ('高知県', 0.59976226),
 ('福井県', 0.59923124)]

In [34]:
#infervec (cos) ['都道府県']のtextと['keyword']の類似度
search_events_doc2vec4('日本三景のひとつに数えられる「宮島」。島内の「嚴島（いつくしま）神社」や「弥山（みせん）原始林」などは、世界文化遺産に登録されています。自然あふれる宮島には野生動物も多く住み、ニホンジカやタヌキ、野鳥などが。また、グルメも宮島観光ではずせません。名物・もみじ饅頭はもちろん、宮島二大グルメの牡蠣とあなごは必食！',events_doc2vec)

[('岡山県', 0.35948125),
 ('佐賀県', 0.351585),
 ('和歌山県', 0.3490944),
 ('滋賀県', 0.3468756),
 ('大分県', 0.34040153),
 ('石川県', 0.33925065),
 ('広島県', 0.33568233),
 ('富山県', 0.33259422),
 ('栃木県', 0.3265802),
 ('愛媛県', 0.32468748)]

In [35]:
#infervec (cos) ['都道府県']のtextと['keyword']の類似度
search_events_doc2vec4('宮島',events_doc2vec)

[('山口県', 0.33176288),
 ('島根県', 0.31834087),
 ('埼玉県', 0.30978042),
 ('佐賀県', 0.30930087),
 ('高知県', 0.30860493),
 ('岡山県', 0.3081013),
 ('香川県', 0.3014824),
 ('愛媛県', 0.29913205),
 ('鹿児島県', 0.29097316),
 ('石川県', 0.29003516)]

In [36]:
search_events_doc2vec4('一番高い山',events_doc2vec)

[('富山県', 0.39477015),
 ('千葉県', 0.38799438),
 ('佐賀県', 0.37810257),
 ('神奈川県', 0.37564507),
 ('福井県', 0.370023),
 ('石川県', 0.3672947),
 ('徳島県', 0.3639512),
 ('岩手県', 0.36338654),
 ('宮城県', 0.3601965),
 ('滋賀県', 0.35897794)]

In [37]:
search_events_cos('日本三景のひとつに数えられる「宮島」。島内の「嚴島（いつくしま）神社」や「弥山（みせん）原始林」などは、世界文化遺産に登録されています。自然あふれる宮島には野生動物も多く住み、ニホンジカやタヌキ、野鳥などが。また、グルメも宮島観光ではずせません。名物・もみじ饅頭はもちろん、宮島二大グルメの牡蠣とあなごは必食！', events)

[('和歌山県', 0.6625362661926),
 ('滋賀県', 0.6384805626941877),
 ('富山県', 0.6365973880096233),
 ('千葉県', 0.6275615161139338),
 ('徳島県', 0.6185360168780255),
 ('大分県', 0.6094136208244431),
 ('三重県', 0.6074473137493471),
 ('長崎県', 0.6054228518955489),
 ('佐賀県', 0.6050011516160212),
 ('広島県', 0.604978857795083)]

In [38]:
search_events_cos('宮島', events)

[('山口県', 0.17801671634528),
 ('広島県', 0.17339790827880117),
 ('長崎県', 0.1596039125914673),
 ('岡山県', 0.13760575044586265),
 ('鹿児島県', 0.13069186990779363),
 ('愛媛県', 0.12977148415195566),
 ('静岡県', 0.12456848175477066),
 ('鳥取県', 0.12400480629610346),
 ('大阪府', 0.12141749128669643),
 ('福岡県', 0.12113238479900283)]

In [39]:
search_events_cos('一番高い山', events)

[('和歌山県', 0.41345843562792367),
 ('富山県', 0.41082443641320565),
 ('千葉県', 0.4090893228687678),
 ('徳島県', 0.39250397876911547),
 ('岐阜県', 0.3902327698777083),
 ('山形県', 0.37756768649636624),
 ('福井県', 0.3721740668908316),
 ('島根県', 0.36511619789388716),
 ('大分県', 0.3633764174100329),
 ('鳥取県', 0.36148247674260203)]

In [40]:
search_events_cca('日本三景のひとつに数えられる「宮島」。島内の「嚴島（いつくしま）神社」や「弥山（みせん）原始林」などは、世界文化遺産に登録されています。自然あふれる宮島には野生動物も多く住み、ニホンジカやタヌキ、野鳥などが。また、グルメも宮島観光ではずせません。名物・もみじ饅頭はもちろん、宮島二大グルメの牡蠣とあなごは必食！', events)

[('福井県', 0.29387607637223695),
 ('徳島県', 0.27905677831421666),
 ('島根県', 0.2770464837074377),
 ('大分県', 0.2768608318340613),
 ('和歌山県', 0.26589784561508284),
 ('京都府', 0.2597812076308172),
 ('熊本県', 0.24992878981520464),
 ('兵庫県', 0.2456875646219469),
 ('富山県', 0.24397605846322673),
 ('鳥取県', 0.24378214607321527)]

In [41]:
search_events_cca('宮島', events)

[('広島県', 0.017622940166339172),
 ('島根県', 0.016369166470055797),
 ('兵庫県', 0.015361581564497457),
 ('山口県', 0.014742884934517465),
 ('鳥取県', 0.013924698244082524),
 ('大分県', 0.013370777288423903),
 ('徳島県', 0.01309388875403509),
 ('長崎県', 0.012475851523800523),
 ('和歌山県', 0.012336500513183087),
 ('愛媛県', 0.01219502796819896)]

In [42]:
search_events_cca('一番高い山', events)

[('徳島県', 0.0506519647024501),
 ('和歌山県', 0.04879440063632084),
 ('福井県', 0.048698571038100824),
 ('京都府', 0.048231774996724014),
 ('島根県', 0.04763214667617136),
 ('大分県', 0.04688616156708705),
 ('兵庫県', 0.046231064655996953),
 ('富山県', 0.043206224215651616),
 ('鳥取県', 0.04317604191696974),
 ('熊本県', 0.042523878773224645)]

In [43]:
model_doc.wv.most_similar('サッカー')

[('フットサル', 0.7777404189109802),
 ('フットボール', 0.7727267146110535),
 ('少年サッカー', 0.7433319091796875),
 ('女子サッカー', 0.7351254224777222),
 ('高校サッカー', 0.726090669631958),
 ('サッカーチーム', 0.7176849842071533),
 ('バレーボール', 0.7045983076095581),
 ('ゴールキーパー', 0.6908824443817139),
 ('Jリーガー', 0.685981273651123),
 ('日本サッカー協会', 0.6841729879379272)]

In [44]:
model.wv.most_similar('サッカー')

[('フットサル', 0.6827920079231262),
 ('日本サッカー協会', 0.6688693761825562),
 ('世界のサッカー', 0.6633233428001404),
 ('後藤健生', 0.6591907739639282),
 ('少年サッカー', 0.6590936779975891),
 ('サッカーチーム', 0.6562634706497192),
 ('フットボール', 0.6452617049217224),
 ('サッカー審判員', 0.6445564031600952),
 ('デットマール・クラマー', 0.643462061882019),
 ('ゴールキーパー', 0.6402513980865479)]

In [45]:
model_doc.docvecs.similarity('サッカー','東京')

0.18747123

In [46]:
model_doc.docvecs.similarity_unseen_docs(model_doc, ['サッカー'], ['東京'], alpha=1, min_alpha=0.0001, steps=5)

0.4418633